In [ ]:
from pycaret.time_series import *
import pandas as pd

# อ่านข้อมูลจากไฟล์ CSV
df = pd.read_csv("province/data_eng_day.csv", parse_dates=["timestamp"], index_col="timestamp")
df = df.drop(columns=["Unnamed: 0"])

# นับจำนวน index ที่ซ้ำกันและแสดงค่าที่ซ้ำ
print(df.index.duplicated().sum())  # นับจำนวน index ที่ซ้ำกัน
print(df.index[df.index.duplicated()])  # แสดงค่าที่ซ้ำ

# ลบ index ที่ซ้ำกันและตั้งค่าความถี่เป็นรายวัน
df = df[~df.index.duplicated(keep="last")]
df = df.asfreq("D")  # ตั้งค่าความถี่เป็นรายวัน
df = df.interpolate()

# ตรวจสอบข้อมูลอีกครั้ง
print("จำนวน index ที่ซ้ำกันหลังจากแก้ไข:", df.index.duplicated().sum())
print("Index เป็น unique หรือไม่:", df.index.is_unique)

# แปลงคอลัมน์ timestamp ให้เป็น DatetimeIndex หากยังไม่ได้ทำ
df.index = pd.to_datetime(df.index)

# ตรวจสอบให้แน่ใจว่าไม่มีคอลัมน์ที่ไม่ต้องการ
df = df[['pm_2_5', 'humidity', 'pm_10', 'temperature']]  # ใช้เฉพาะคอลัมน์ที่ต้องการ

# แบ่งข้อมูลเป็น train set และ test set
train_size = int(len(df)) - 7
train_df = df.iloc[:train_size]  # ใช้ข้อมูลก่อนหน้าทั้งหมดเป็น train set
test_df = df.iloc[train_size:]  
train_df.index = train_df.index.to_period("D")
test_df.index = test_df.index.to_period("D")
print(len(train_df), len(test_df))

# ตั้งค่า PyCaret และทำ differencing ข้อมูล
exp = TSForecastingExperiment()
exp.setup(data=train_df, target='pm_10', session_id=123, transform_target=True)

# แสดงข้อมูลที่ผ่านการ differencing แล้ว
differenced_data = exp.get_config('X')
print(differenced_data)